# Requirements

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import datetime
from incident_parser import IncidentParser
from config_reader import read_config
import markdown
import os
from pathlib import Path
import utils

# Configuration

In [3]:
config = read_config('../config/config.yml')

In [4]:
incident_dirs = config['incident_dirs']
template_dir = config['template_dir']
build_dir = config['build_dir']

In [5]:
no_news_message = config['no_news_message']

In [6]:
current_overview_columns = ['end_date', 'affected', 'title', 'level']
planned_overview_columns = ['start_date', 'end_date', 'affected', 'title', 'level']

In [7]:
meta_data_fields = ['title', 'start_date', 'end_date', 'affected', 'level', 'planned',]
affected = ['tier1_compute', 'tier1_data', 'tier1_cloud',
            'tier2_antwerp', 'tier2_brussels', 'tier2_gent', 'tier2_leuven',
            'vsc_backbone']
level = ['low', 'medium', 'high']
sep = r'\s*,\s*'

In [8]:
os.chdir('/home/gjb/Projects/github_actions_test/')

# Incidents

In [9]:
parser = IncidentParser(meta_data_fields=meta_data_fields, affected=affected, level=level,
                        sep=sep, is_verbose=True, is_permissive=True)

In [10]:
for dir_path in incident_dirs:
    parser.parse(dir_path)

parsing directory 'incidents'
parsing file 'incidents/indicent_04.yml'
validating file 'incidents/indicent_04.yml'
parsing file 'incidents/indicent_01.yml'
validating file 'incidents/indicent_01.yml'
parsing file 'incidents/indicent_02.yml'
validating file 'incidents/indicent_02.yml'
### error: incidents/indicent_02.yml: invalid level value intermediate
parsing file 'incidents/indicent_03.yml'
validating file 'incidents/indicent_03.yml'


In [11]:
incidents = parser.incidents

In [12]:
incidents

,title,start_date,end_date,affected,level,planned,updated,id,content
0,login nodes down,2022-01-20 09:00:00,NaT,tier2_leuven,high,False,2022-02-25 12:14:32+01:00,incident_0001,The login nodes seem to be down. \n
1,update of storage system software,2022-01-20 09:00:00,2022-01-21 13:00:00,tier2_leuven,high,True,2022-02-08 09:58:53+01:00,incident_0002,The drivers for the parallel file system need ...
2,Belnet maintenance,2022-01-22 11:30:00,2022-01-22 13:00:00,vsc_backbone,high,True,2022-03-20 08:04:25+01:00,incident_0003,The Belnet VSC backbone will be down for maint...
3,Belnet maintenance,2022-01-22 11:30:00,2022-01-22 13:00:00,tier1_compute,high,True,2022-03-20 08:04:25+01:00,incident_0004,The Belnet VSC backbone will be down for maint...


In [11]:
incidents.end_date.fillna(datetime.datetime.fromisoformat('2030-12-31 00:00:00'), inplace=True)

In [12]:
incidents

,title,start_date,end_date,affected,level,planned,content
0,login nodes down,2022-01-20 09:00:00,2030-12-31 00:00:00,tier2_leuven,high,False,The login nodes seem to be down. \n
1,update of storage system software,2022-01-20 09:00:00,2022-01-21 13:00:00,tier2_leuven,high,True,The drivers for the parallel file system need ...
2,Belnet maintenance,2022-01-22 11:30:00,2022-01-22 13:00:00,vsc_backbone,high,True,The Belnet VSC backbone will be down for maint...
3,Belnet maintenance,2022-01-22 11:30:00,2022-01-22 13:00:00,tier1_compute,high,True,The Belnet VSC backbone will be down for maint...


In [13]:
incidents.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4 entries, 0 to 3
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype                    
---  ------      --------------  -----                    
 0   title       4 non-null      object                   
 1   start_date  4 non-null      datetime64[ns]           
 2   end_date    3 non-null      datetime64[ns]           
 3   affected    4 non-null      object                   
 4   level       4 non-null      object                   
 5   planned     4 non-null      bool                     
 6   updated     4 non-null      datetime64[ns, UTC+01:00]
 7   id          4 non-null      object                   
 8   content     4 non-null      object                   
dtypes: bool(1), datetime64[ns, UTC+01:00](1), datetime64[ns](2), object(5)
memory usage: 388.0+ bytes


In [16]:
incidents['href'] = incidents['affected'] + '.html#' + incidents['id']

In [17]:
incidents

,title,start_date,end_date,affected,level,planned,updated,id,content,href
0,login nodes down,2022-01-20 09:00:00,NaT,tier2_leuven,high,False,2022-02-25 12:14:32+01:00,incident_0001,The login nodes seem to be down. \n,tier2_leuven.html#incident_0001
1,update of storage system software,2022-01-20 09:00:00,2022-01-21 13:00:00,tier2_leuven,high,True,2022-02-08 09:58:53+01:00,incident_0002,The drivers for the parallel file system need ...,tier2_leuven.html#incident_0002
2,Belnet maintenance,2022-01-22 11:30:00,2022-01-22 13:00:00,vsc_backbone,high,True,2022-03-20 08:04:25+01:00,incident_0003,The Belnet VSC backbone will be down for maint...,vsc_backbone.html#incident_0003
3,Belnet maintenance,2022-01-22 11:30:00,2022-01-22 13:00:00,tier1_compute,high,True,2022-03-20 08:04:25+01:00,incident_0004,The Belnet VSC backbone will be down for maint...,tier1_compute.html#incident_0004


In [ ]:
incidents.to_html(

# Testing

In [14]:
now = datetime.datetime.fromisoformat('2022-01-20 10:00:00')

In [15]:
utils.select_current_incidents(incidents, date=now)

,title,start_date,end_date,affected,level,planned,content
0,login nodes down,2022-01-20 09:00:00,2030-12-31 00:00:00,tier2_leuven,high,False,The login nodes seem to be down. \n
1,update of storage system software,2022-01-20 09:00:00,2022-01-21 13:00:00,tier2_leuven,high,True,The drivers for the parallel file system need ...


In [16]:
utils.select_planned_maintenance(incidents, date=now)

,title,start_date,end_date,affected,level,planned,content
2,Belnet maintenance,2022-01-22 11:30:00,2022-01-22 13:00:00,vsc_backbone,high,True,The Belnet VSC backbone will be down for maint...
3,Belnet maintenance,2022-01-22 11:30:00,2022-01-22 13:00:00,tier1_compute,high,True,The Belnet VSC backbone will be down for maint...


# Generation

In [17]:
current_incidents = utils.select_current_incidents(incidents, date=now)

In [18]:
current_table = utils.dataframe_to_html_table(
    current_incidents[current_overview_columns],
    no_news_message
)

In [19]:
planned_maintenance = utils.select_planned_maintenance(incidents, date=now)

In [20]:
planned_table = utils.dataframe_to_html_table(
    planned_maintenance[planned_overview_columns],
    no_news_message
)

In [21]:
utils.process_template(template_dir / 'index.html', build_dir,
                       {'overview_table_current': current_table,
                        'overview_table_planned': planned_table})

In [22]:
incidents.groupby('affected').count()

,title,start_date,end_date,level,planned,content
affected,,,,,,
tier1_compute,1,1,1,1,1,1
tier2_leuven,2,2,2,2,2,2
vsc_backbone,1,1,1,1,1,1


In [23]:
print(markdown.markdown(incidents.loc[0].content))

<p>The login nodes seem to be down. </p>


In [24]:
current_incidents

,title,start_date,end_date,affected,level,planned,content
0,login nodes down,2022-01-20 09:00:00,2030-12-31 00:00:00,tier2_leuven,high,False,The login nodes seem to be down. \n
1,update of storage system software,2022-01-20 09:00:00,2022-01-21 13:00:00,tier2_leuven,high,True,The drivers for the parallel file system need ...


In [25]:
current_incidents.loc[:, ['end_date']] = current_incidents.end_date.apply(lambda x: str(x) if x < datetime.datetime.fromisoformat('2022-12-31 00:00:00') else '')

In [26]:
current_incidents

,title,start_date,end_date,affected,level,planned,content
0,login nodes down,2022-01-20 09:00:00,,tier2_leuven,high,False,The login nodes seem to be down. \n
1,update of storage system software,2022-01-20 09:00:00,2022-01-21 13:00:00,tier2_leuven,high,True,The drivers for the parallel file system need ...


In [27]:
current_incidents.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2 entries, 0 to 1
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   title       2 non-null      object        
 1   start_date  2 non-null      datetime64[ns]
 2   end_date    2 non-null      object        
 3   affected    2 non-null      object        
 4   level       2 non-null      object        
 5   planned     2 non-null      bool          
 6   content     2 non-null      object        
dtypes: bool(1), datetime64[ns](1), object(5)
memory usage: 114.0+ bytes
